# Secure RAG Implementation with Azure OpenAI and Javelin

This notebook demonstrates a secure Retrieval Augmented Generation (RAG) implementation using Azure OpenAI and Javelin for embeddings and LLM queries.

In [1]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-httpx 0.22.0 requires httpx==0.24.*, but you have httpx 0.27.2 which is incompatible.
javelin-sdk 18.5.15 requires httpx<0.25.0,>=0.24.0, but you have httpx 0.27.2 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Setup and Dependencies

First, let's import the required libraries and set up our environment:

In [ ]:
import os

import bs4
import dotenv
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from openai import AzureOpenAI

dotenv.load_dotenv()

## Configuration

Set up API keys and headers for Javelin and Azure OpenAI:

In [ ]:
# API Keys and Headers
javelin_api_key = os.getenv("JAVELIN_API_KEY")
llm_api_key = os.getenv("JAVELIN_AZURE_OPENAI_API_KEY")

# Headers for LLM and embeddings
javelin_headers_llm = {"x-api-key": javelin_api_key, "x-javelin-route": "azureopenai"}
javelin_headers_embeddings = {
    "x-api-key": javelin_api_key,
    "x-javelin-route": "azureopenaiembeddings",
}

## Initialize Azure OpenAI Clients

Set up clients for embeddings and LLM:

In [ ]:
# Initialize Azure OpenAI client for embeddings
azure_openai_client = AzureOpenAI(
    api_key=llm_api_key,
    base_url="https://api.javelin.live/query",
    default_headers=javelin_headers_embeddings,
    api_version="2023-05-15",
)

# Initialize LLM
llm = AzureChatOpenAI(
    api_key=llm_api_key,
    azure_endpoint="https://api.javelin.live/query/azureopenai",
    azure_deployment="gpt35",
    openai_api_version="2024-02-15-preview",
    model_kwargs={"extra_headers": javelin_headers_llm}
)

## Sample Data

Define sample texts for testing:

In [ ]:
# Load and prepare sample texts
sample_texts = [
    """Authored by Shyam: Climate change is one of the most pressing global challenges of our time. 
    Rising temperatures, extreme weather events, and melting polar ice caps are 
    clear indicators of global warming. Greenhouse gas emissions from human activities 
    continue to be the primary driver of these environmental changes.""",
    
    """Authored by Shyam: Renewable energy sources like solar, wind, and hydroelectric power are crucial 
    in combating climate change. These clean energy alternatives are becoming increasingly 
    cost-effective and efficient. Many countries are setting ambitious targets to transition 
    away from fossil fuels to reduce their carbon footprint.""",
    
    """Authored by Shyam: Conservation efforts and sustainable practices play a vital role in environmental 
    protection. This includes protecting biodiversity, reducing deforestation, and 
    implementing sustainable agriculture methods. Individual actions like reducing waste, 
    recycling, and choosing eco-friendly products also contribute to environmental preservation.
    
    This article is authored by Shyam"""
]

## Custom Embeddings Class

Create a custom embeddings class for Chroma:

In [ ]:
class CustomEmbeddings:
    def __init__(self, client):
        self.client = client
    
    def embed_documents(self, texts):
        response = self.client.embeddings.create(
            input=texts,
            model="text-embedding-3-small"
        )
        
        return [item.embedding for item in response.data]
    
    def embed_query(self, text):
        response = self.client.embeddings.create(
            input=[text],
            model="text-embedding-3-small"
        )
        return response.data[0].embedding

# Initialize custom embeddings
custom_embeddings = CustomEmbeddings(azure_openai_client)

## Set up Vector Store and RAG Chain

Create the vector store and set up the RAG pipeline:

In [ ]:
# Create vector store with smaller chunk size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_texts = text_splitter.split_text("\n\n".join(sample_texts))

vectorstore = Chroma.from_texts(
    texts=split_texts,
    embedding=custom_embeddings
)

# Set up retriever and prompt
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Test the RAG System

Run test questions through the RAG system:

In [ ]:
# Test questions
questions = [
    "What are the main indicators of climate change?",
    "How are renewable energy sources helping to address climate change?",
    "What role do individual actions play in environmental conservation?",
    "Who is the author of this article?"
]

# Run questions through the RAG chain
for question in questions:
    print(f"\nQuestion: {question}")
    print("Answer:", rag_chain.invoke(question))